# Large problem with 1000 assets and 100 factors

In [1]:
import numpy as np
import pandas as pd
import cvxpy as cvx

from cvx.risk.factor import FactorModel
from cvx.portfolio.min_risk import minrisk_problem
from util.random import random_factors, random_beta, random_noise, random_assets

In [2]:
T = 2000
factors = random_factors(T=T, N=100, const_factor=False)

In [3]:
beta = random_beta(assets=random_assets(1000), factors=factors)

In [4]:
ret = factors @ beta + 0.01 * random_noise(factors @ beta)

In [5]:
triangle = FactorModel(assets=len(ret.columns), k=100)

In [6]:
#w = triangle.weights
w = cvx.Variable(1000)
y = cvx.Variable(100)

problem = minrisk_problem(triangle, w, y=y)

#objective = cvx.Minimize(triangle.estimate(w, y=y))
#constraints = [w >= 0, cvx.sum(w) == 1] + triangle.bounds.constraints(w)

triangle.update(exposure=beta.values, 
                cov=factors.cov().values, 
                idiosyncratic_risk=pd.DataFrame(data=ret - factors @ beta, index=ret.index, columns=ret.columns).std().values,
                lower_assets=np.zeros(1000),
                upper_assets=np.ones(1000),
                lower_factors=-0.1*np.ones(100),
                upper_factors= 0.1*np.ones(100))



In [7]:
%%timeit
for i in range(1):
    #objective = cvx.Minimize(triangle.estimate(w, y=y))
    #constraints = [w >= 0, cvx.sum(w) == 1] + triangle.bounds.constraints(w)

    problem = minrisk_problem(triangle, w, y=y)
    triangle.update(exposure=beta.values, 
                         cov=factors.cov().values, 
                         idiosyncratic_risk=pd.DataFrame(data=ret - factors @ beta, index=ret.index, columns=ret.columns).std().values,
                         lower_assets=np.zeros(1000),
                         upper_assets=np.ones(1000),
                         lower_factors=-0.1*np.ones(100),
                         upper_factors= 0.1*np.ones(100))
    
    problem.solve(ignore_dpp=True)

150 ms ± 940 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
